In [8]:
import spacy
import nltk
import pandas as pd
import sqlite3 as sql
from pytrends.request import TrendReq
from nlp_helper import *
from config_new import config
import json
nlp = spacy.load('en_core_web_trf')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-d

True

#### SQL connections

In [3]:
con = sql.connect(config['database'])
cur = con.cursor()

## NLP Vadar and Bags

In [ ]:
id_list = pd.read_sql_query('SELECT * FROM Front_Page_text ',con)
id_list = id_list['id'].tolist()
for idnum in id_list:
    try:
        row = pd.read_sql_query(f"SELECT * FROM Front_Page_text WHERE id = {idnum} ",con)
        idx = row['id'].values[0]
        text = row['front_page_text'].values[0]
        p_tag_block = remove_stop_words(text)
        p_tag_block = processed_feature(' '.join(p_tag_block))
        p_vocab = create_bag_of_words(p_tag_block)
        p_Tfidf = Tfidf([p_tag_block])
        p_phrases = create_bag_of_phrases([p_tag_block], n_gram_range=(2,4))
        dict_Tfidf = {key:value for key, value in p_Tfidf}
        dict_Tfidf = dict(sorted(dict_Tfidf.items(),key = lambda x:x[1], reverse = True))
        avg = sum(dict_Tfidf.values())/len(dict_Tfidf.keys())
        dict_Tfidf_insert = {}
        for key, value in dict_Tfidf.items():
            if value > avg:
                dict_Tfidf_insert[key] =value 
        dict_vocab_insert = dict(nltk.FreqDist(p_vocab).most_common(len(dict_Tfidf_insert.keys())))
        vadar_out = vadar(p_tag_block)
        cur.execute(f"""INSERT INTO Front_Page_vader (front_page_text_id , sediment_neg , sediment_neu , sediment_pos , sediment_comp ) VALUES ({idx},{vadar_out['pos']},{vadar_out['neu']},{vadar_out['neg']},{vadar_out['comp']})""")
        cur.execute(f"""INSERT INTO Front_Page_bag (front_page_text_id , bag_a_words_json , Tfidf_json, Phrase_List ) VALUES ({idx},'{json.dumps(dict_vocab_insert)}','{json.dumps(dict_Tfidf_insert)}','{','.join(p_phrases)}')""")
        con.commit()
    except Exception as e:
        continue

## NLP Named Enity Recongiztion 

In [ ]:
id_list = pd.read_sql_query("SELECT id FROM Front_Page_text WHERE id NOT IN (SELECT front_page_text_id FROM Front_Page_nea) ",con)
id_list = id_list['id'].tolist()
for idnum in id_list:
    try:
        row = pd.read_sql_query(f"SELECT * FROM Front_Page_text WHERE id = {idnum} ",con)
        text = nlp(row['front_page_text'].values[0])
        idx = row['id'].values[0]
        for ent in text.ents:
            cur.execute(f"""INSERT INTO Front_Page_nea VALUES ({idx}, '{ent.text}', '{ent.start_char}', '{ent.end_char}', '{ent.label_}')""")
            con.commit()
    except Exception as e:
        continue

#### SQL Connection

In [ ]:
con.close()